# Cálculo de area de agua superficial
Este código utiliza imágenes sentinel para encontrar el área lagunar en un área determinada a lo largo de un periodo de tiempo.


>Creador: **Borja Mir**
>Fecha: **20/04/2021**

In [1]:
import matplotlib.pyplot as plt
import ee 
import folium
# import geehydro
import os
import geemap
import ee.mapclient
import numpy as np
import datetime 
from IPython.display import Image
import pandas as pd
import calendar
from sklearn.linear_model import LinearRegression
import scipy.stats
import scipy
import seaborn as sns
#Esto sirve para que los plots se abran en una ventana aparte
%matplotlib qt 

ee.Initialize()
mapa = geemap.Map(location=[-23.40, -68.23], zoom_start=9.5) # Aquí se determina donde se abre el mapa
mapa

Map(center=[-23.4, -68.23], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(childr…

In [2]:
#area = ee.FeatureCollection("users/borjamirc/La_Brava") #Aquí linkealo a tu colección de earth engine en la ventana de assets, yo hago los shapes en qgis y los subo
area = ee.FeatureCollection(mapa.draw_last_feature) # O aquí te toma la ultima figura dibujada en el mapa que se abrío en la entrada anterior

In [3]:


sentinel = ee.ImageCollection("COPERNICUS/S2_SR")


sentinel = (sentinel
                        .filterBounds(area)
                        .filterDate('2019-01-01', '2021-3-31')
                        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',30))# Filtro por porcentaje de nubosidad           
           )



In [4]:
def NDWIsentinel (image):
    return image.normalizedDifference(['B3', 'B8']).gt(0) 
# .gt(0) devuelve una imagen booleana con 0 donde el valor es menor igual que 0 y 1 si es mayor (greater than 0)


def mNDWIsentinel(image):  
    return image.normalizedDifference(['B3', 'B11']).gt(0)  #sentinel red=3 nir=4 swir2=7  l8 red=4 nir=5 swir2=7  sen2 red=4 nir=8 swir2=12  


def TCWsentinel(image):
    tcw = image.expression(
    '0.1509 * BLUE + 0.1973 * GREEN + 0.3279 * RED + 0.3406 * NIR -0.7112 * SWIR1 - 0.4572 * SWIR2', {
      'NIR': image.select('B5'),
      'RED': image.select('B4'),
      'GREEN': image.select('B3'),
      'BLUE': image.select('B2'),
      'SWIR1': image.select('B11'),
      'SWIR2': image.select('B12')
    })
    return tcw.gt(0)


def EVIsentinel(image):
    evi = image.expression(
    '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))', {
      'NIR': image.select('B5'),
      'RED': image.select('B4'),
      'BLUE': image.select('B2')
    })
    return evi

def NDVIsentinel(image):
    return image.normalizedDifference(['B8', 'B4'])

mNDWI=sentinel.map(mNDWIsentinel)
TCW=sentinel.map(TCWsentinel)
EVI=sentinel.map(EVIsentinel)
NDWI=sentinel.map(NDWIsentinel)
NDVI=sentinel.map(NDVIsentinel)
#map se usa para mapear una función sobre una colección de imágenes, ya que las funciones usan .addBands() se le agregan a la colección como banda nueva por imágen.

In [5]:
#Utilizo mediana envés de promedio para visualizar, así evito los valores anómalos (esto calcula el valor por pixel para toda la serie)
NDWImed = NDWI.median()
mNDWImed = mNDWI.median()
TCWmed = TCW.median()
EVImed = EVI.median()
NDVImed = NDVI.median()

imagen = sentinel.median()  
image_viz_params = {
    'bands': ['B4', 'B3', 'B2'], #agua:b11 b6 b3 o b7 b5 b2,  vegetacion: B7 B3 B2, normal: b4 b3 b2 
    'min': 0,
    'max': 6000, #Con este ajusta el brillo
    'gamma': [1, 1, 1]
}


mapa.addLayer(NDWImed,name='NDWI')
mapa.addLayer(mNDWImed,name='mNDWI')
mapa.addLayer(TCWmed,name='TCW')
mapa.addLayer(NDVImed,name='NDVI')
mapa.addLayer(imagen, image_viz_params, 'Color composite')




mapa

Map(bottom=1257261.0, center=[-33.65135119453337, -70.04735803010178], controls=(WidgetControl(options=['posit…

In [6]:
#El Reducer reduce un area a un valor, aquí yo uso sum() ya que estoy contando los pixeles con agua
#para NDVI se puede usa mean() o el mismo sum() si en la función arriba lo dejas con un .gt(0.3) por ejemplo como te dije en el mail
def area_sum(image):
    dict = image.reduceRegion(ee.Reducer.sum(), area)
    return image.set(dict)

def area_mean(image):
    dict = image.reduceRegion(ee.Reducer.mean(), area)
    return image.set(dict)

## Esta parte es media jodida por que no he podido elegir yo el nombre de la banda que se genera, entonces escribo por ejemplo
## NDWI.getInfo() y veo el nombre de la banda

##Genero la serie de tiempo con los valores

area_mNDWI = mNDWI.select(['nd']).map(area_sum).aggregate_array('nd').getInfo()
area_NDWI = NDWI.select(['nd']).map(area_sum).aggregate_array('nd').getInfo()
area_TCW = TCW.select(['constant']).map(area_sum).aggregate_array('constant').getInfo()
area_NDVI = NDVI.select(['nd']).map(area_mean).aggregate_array('nd').getInfo()

In [7]:
#Armo el dataframe de pandas para mejor manejo
data=pd.DataFrame()

data['Timestamp']=sentinel.aggregate_array('system:time_start').getInfo()
data['Fecha']=[datetime.datetime.fromtimestamp(x//1000) for x in data['Timestamp']]

data['NDWI']=np.asarray(area_NDWI)*10*10/1000000 #De pixeles a km^2
data['mNDWI']=np.asarray(area_mNDWI)*10*10/1000000
data['TCW']=np.asarray(area_TCW)*10*10/1000000

data['NDVI']=np.asarray(area_NDVI)

data=data.sort_values(by=['Fecha'])
data

,Timestamp,Fecha,NDWI,mNDWI,TCW,NDVI
1,1546353737000,2019-01-01 11:42:17,3.094179,5.748778,5.032689,0.042939
0,1546353740000,2019-01-01 11:42:20,5.772588,8.323716,7.356176,0.038044
3,1546613533000,2019-01-04 11:52:13,2.942497,4.918094,4.258642,0.098065
2,1546613537000,2019-01-04 11:52:17,5.777008,8.571829,7.436984,0.089756
5,1547217738000,2019-01-11 11:42:18,3.599908,6.312861,5.298143,0.038705
...,...,...,...,...,...,...
399,1616597534326,2021-03-24 11:52:14,5.450495,7.323405,4.215238,0.105800
398,1616597538086,2021-03-24 11:52:18,8.746057,11.798193,10.029473,0.051736
400,1616769740983,2021-03-26 11:42:20,8.539056,11.277893,9.535385,0.067359
402,1617029533046,2021-03-29 11:52:13,4.569856,5.461043,3.146482,0.110595


In [8]:


fig, axs=plt.subplots(4,1,sharex=True)
axs[0].plot(data['Fecha'],data['NDWI'], label='NDWI')
axs[0].legend()

axs[1].plot(data['Fecha'],data['mNDWI'], label='mNDWI')
axs[1].legend()

axs[2].plot(data['Fecha'],data['TCW'],label='TCW')
axs[2].legend()

axs[3].plot(data['Fecha'],data['NDVI'],label='NDVI')
axs[3].legend()


In [10]:
list(data.columns)

['Timestamp', 'Fecha', 'NDWI', 'mNDWI', 'TCW', 'NDVI']